## 进阶RAG检索   Contextual Compression + Filtering  

### 前期准备

In [8]:
! pip install langchain openai chromadb tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.8 MB/s eta 0:00:0000:0100:01


In [9]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma


# Load pdf
loader = PyPDFLoader("https://arxiv.org/pdf/2309.10305v2.pdf")
data = loader.load()

# Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
splits = text_splitter.split_documents(data[:6])

import os
from getpass import getpass

OPENAI_API_KEY = getpass()

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# VectorDB
embedding = OpenAIEmbeddings()
vectordb = Chroma.from_documents(documents=splits, embedding=embedding)

retriever = vectordb.as_retriever()

 ························································


In [11]:
base_docs = retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

base_docs

[Document(page_content='In this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,\nBaichuan 2 achieves significant improvements over', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf'}),
 Document(page_content='demonstrates strong performance on medical and\nlegal domain tasks. On benchmarks such as\nMedQA (Jin et al., 2021) and JEC-QA (Zhong\net al., 2020), Baichuan 2 outperforms other open-\nsource models, making it a suitable foundation\nmodel for domain-specific optimization.\nAdditionally, we also released two chat\nmodels, Baichuan 2-7B-Chat and Baichuan 2-\n13B-Chat, optimized to follow 

#### Contextual Extractor

In [14]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0)
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

In [15]:
compressed_docs

[Document(page_content='Baichuan 2, a series of large-scale multilingual language models. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters.', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf'}),
 Document(page_content='Baichuan 2 outperforms other open-source models, making it a suitable foundation model for domain-specific optimization.', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf'}),
 Document(page_content='Baichuan 1. On general benchmarks like MMLU (Hendrycks et al., 2021a), CMMLU (Li et al., 2023), and C-Eval (Huang et al., 2023), Baichuan 2-7B achieves nearly 30% higher performance compared to Baichuan 1-7B. Specifically, Baichuan 2 is optimized to improve performance on math and code problems. On the GSM8K (Cobbe et al., 2021) and HumanEval (Chen et al., 2021) evaluations, Baichuan 2 nearly doubles the results of the Baichuan 1.', metadata={'page': 1, 'sou

#### Contextual filters

##### LLMChainFilter

In [16]:
from langchain.retrievers.document_compressors import LLMChainFilter

_filter = LLMChainFilter.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

compressed_docs

[Document(page_content='In this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,\nBaichuan 2 achieves significant improvements over', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf'}),
 Document(page_content='demonstrates strong performance on medical and\nlegal domain tasks. On benchmarks such as\nMedQA (Jin et al., 2021) and JEC-QA (Zhong\net al., 2020), Baichuan 2 outperforms other open-\nsource models, making it a suitable foundation\nmodel for domain-specific optimization.\nAdditionally, we also released two chat\nmodels, Baichuan 2-7B-Chat and Baichuan 2-\n13B-Chat, optimized to follow 

##### EmbeddingsFilter

In [19]:
from langchain.retrievers.document_compressors import EmbeddingsFilter

embeddings_filter = EmbeddingsFilter(embeddings=embedding, similarity_threshold=0.76)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=embeddings_filter, base_retriever=retriever
)

compressed_docs = compression_retriever.get_relevant_documents(
    "What is baichuan2 ？"
)

In [20]:
compressed_docs

[_DocumentWithState(page_content='In this technical report, we introduce Baichuan\n2, a series of large-scale multilingual language\nmodels. Baichuan 2 has two separate models,\nBaichuan 2-7B with 7 billion parameters and\nBaichuan 2-13B with 13 billion parameters. Both\nmodels were trained on 2.6 trillion tokens, which\nto our knowledge is the largest to date, more than\ndouble that of Baichuan 1 (Baichuan, 2023b,a).\nWith such a massive amount of training data,\nBaichuan 2 achieves significant improvements over', metadata={'page': 1, 'source': 'https://arxiv.org/pdf/2309.10305v2.pdf'}, state={'embedded_doc': [-0.014723360590013786, 0.0002893662431127455, 0.011971203826286712, -0.0010278220838435682, 0.0057991874664249056, 0.01773310905529369, -0.027819831124549266, -0.003948599329826443, -0.012045770163767604, -0.03367663718691621, -0.0003272846611226113, 0.026965713379761894, -0.0192786555393206, -0.002374243547828515, 0.012676189827434818, 0.0022115545422806845, 0.01070358540233674